<a href="https://colab.research.google.com/github/Behnam9473/AI/blob/main/custom_fuction_calling_1403_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForQuestionAnswering, AutoTokenizer,BertForSequenceClassification
import torch

In [ ]:
drive.mount('/content/drive')  # This mounts your Drive to a local directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/Nojan/ND/MVP/db_entity.xlsx'
df = pd.read_excel(file_path)

In [ ]:
print(df.head())


                                     question       intent      entities  \
0                 مقدار فروش دیروز چقدر بوده؟  fetch_sales    time_range   
1       چه تعداد از محصولات امروز فروخته شده؟  fetch_sales    time_range   
2       چه تعداد از محصولات امروز فروخته اند؟  fetch_sales    time_range   
3      چه محصولاتی بیشترین فروش را داشته‌اند؟  fetch_sales  product_name   
4   چه محصولی کمترین فروش را امروز داشته است؟  fetch_sales  product_name   

       value  
0  1 day ago  
1      today  
2      today  
3  max_sales  
4  min_sales  


In [ ]:
model_path ="/content/drive/MyDrive/my_qa_model"
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.25)

In [ ]:
def encode_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_data = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            return_attention_mask=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0), torch.tensor(labels)

In [ ]:
intent_labels = df['intent'].unique()
intent_label_map = {label: idx for idx, label in enumerate(intent_labels)}
# df['intent_label'] = df['intent'].map(intent_label_map)


In [ ]:
len(intent_label_map)

11

In [ ]:
# train_inputs, train_masks, train_labels = encode_data(train_df['question'], [intent_labels[i] for i in train_df['intent']])
# val_inputs, val_masks, val_labels = encode_data(val_df['question'], [intent_labels[i] for i in val_df['intent']])
# test_inputs, test_masks, test_labels = encode_data(test_df['question'], [intent_labels[i] for i in test_df['intent']])


In [ ]:
# Change how train_labels, val_labels, and test_labels are generated
train_labels = [intent_label_map[intent] for intent in train_df['intent']] # Use intent_label_map for mapping
val_labels = [intent_label_map[intent] for intent in val_df['intent']]    # Use intent_label_map for mapping
test_labels = [intent_label_map[intent] for intent in test_df['intent']]   # Use intent_label_map for mapping

# Now call encode_data with the corrected labels
train_inputs, train_masks, train_labels = encode_data(train_df['question'], train_labels)
val_inputs, val_masks, val_labels = encode_data(val_df['question'], val_labels)
test_inputs, test_masks, test_labels = encode_data(test_df['question'], test_labels)

In [ ]:
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(intent_label_map))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/my_qa_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# مرحله 5: تنظیم مدل برای GPU در صورت موجود بودن
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# مرحله 6: تعریف optimizer و loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# مرحله 7: آموزش مدل (Training loop)
epochs = 10
batch_size = 16

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(0, len(train_inputs), batch_size):
        input_batch = train_inputs[i:i+batch_size].to(device)
        mask_batch = train_masks[i:i+batch_size].to(device)
        label_batch = train_labels[i:i+batch_size].to(device)

        model.zero_grad()

        outputs = model(input_batch, attention_mask=mask_batch, labels=label_batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_inputs)}")

Epoch 1/10 | Loss: 0.11827328841980189
Epoch 2/10 | Loss: 0.044879818411722575
Epoch 3/10 | Loss: 0.02450189440081653
Epoch 4/10 | Loss: 0.010961495050692667
Epoch 5/10 | Loss: 0.007263494995699081
Epoch 6/10 | Loss: 0.007147983856261048
Epoch 7/10 | Loss: 0.0068528653126737294
Epoch 8/10 | Loss: 0.004791278128414393
Epoch 9/10 | Loss: 0.004790461333000769
Epoch 10/10 | Loss: 0.008424636597434679


In [ ]:
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# مرحله 8: ارزیابی مدل روی داده‌های تست
with torch.no_grad():
    for i in range(0, len(test_inputs), batch_size):
        input_batch = test_inputs[i:i+batch_size].to(device)
        mask_batch = test_masks[i:i+batch_size].to(device)

        outputs = model(input_batch, attention_mask=mask_batch)
        preds = torch.argmax(outputs.logits, dim=1)

        # نمایش پیش‌بینی‌ها
        print(f"Predictions: {preds}")

Predictions: tensor([7, 4, 3, 1, 1, 0, 5, 5, 8, 2, 5, 0, 2, 3, 7, 2], device='cuda:0')
Predictions: tensor([ 3,  1, 10,  5,  5,  3,  0,  4,  4, 10,  3,  5,  3,  4,  3,  3],
       device='cuda:0')
Predictions: tensor([ 3, 10,  2,  3, 10,  5,  0,  6,  5,  2,  9,  1,  0,  6,  0,  1],
       device='cuda:0')
Predictions: tensor([ 5,  4,  6,  4,  2,  0,  2,  9,  2,  2,  3,  0,  0,  7, 10, 10],
       device='cuda:0')
Predictions: tensor([ 4,  0,  2,  3,  0,  0,  0, 10,  9,  4,  1,  0,  3,  3,  1,  4],
       device='cuda:0')
Predictions: tensor([ 6, 10,  2,  0,  1,  2,  0,  3,  6,  0,  7,  4,  5,  0,  7,  5],
       device='cuda:0')
Predictions: tensor([10,  3,  3,  1,  7,  3,  3,  0,  0,  8,  3,  1,  3,  0,  4,  0],
       device='cuda:0')
Predictions: tensor([ 0,  0,  1,  0,  3,  2,  1,  2,  1,  4,  7,  7,  3,  0, 10,  6],
       device='cuda:0')
Predictions: tensor([0, 0, 3, 8, 7, 1, 7, 4, 5, 3, 3, 8, 4, 4, 0, 3], device='cuda:0')
Predictions: tensor([5, 6, 3], device='cuda:0')


In [ ]:
# ذخیره مدل و tokenizer
model.save_pretrained('./my_model_function_calling_v0.0.1')
tokenizer.save_pretrained('./my_model_function_calling_v0.0.1')


('./my_model_function_calling_v0.0.1/tokenizer_config.json',
 './my_model_function_calling_v0.0.1/special_tokens_map.json',
 './my_model_function_calling_v0.0.1/vocab.txt',
 './my_model_function_calling_v0.0.1/added_tokens.json',
 './my_model_function_calling_v0.0.1/tokenizer.json')

In [ ]:
label_map = {
    'fetch_sales': 0,
    'fetch_inventory': 1,
    'fetch_orders': 2,
    'fetch_best_selling_product': 3,
    'fetch_total_revenue': 4,
    'fetch_product_sales_count': 5,
    'fetch_customers': 6,
    'fetch_revenue': 7,
    'fetch_delivery': 8,
    'fetch_low_stock': 9,
    'fetch_profit': 10
}

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# بارگذاری مدل ذخیره شده
model_path = './my_model_function_calling_v0.0.1'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=11)

# تنظیم مدل برای استفاده از GPU در صورت موجود بودن
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# لیست پرامپت‌ها برای تست
prompts = [
    "سود امروز چقدر بوده است؟",
    "کدام محصولات به زودی تمام می‌شوند؟",
    "چه تعداد سفارشات امروز دریافت شده است؟",
    "کدام محصول بیشترین فروش را داشته است؟",
    "درآمد کل این ماه چقدر بوده است؟"
]

# پیش‌بینی intent
model.eval()  # تنظیم مدل به حالت ارزیابی
for prompt in prompts:
    # توکنایز کردن ورودی
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    with torch.no_grad():
        # پیش‌بینی خروجی مدل
        outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()

    # یافتن نام intent بر اساس label_map
    intent = [key for key, value in label_map.items() if value == predicted_label][0]

    print(f"Prompt: {prompt}")
    print(f"Predicted Intent: {intent}\n")


Prompt: سود امروز چقدر بوده است؟
Predicted Intent: fetch_profit

Prompt: کدام محصولات به زودی تمام می‌شوند؟
Predicted Intent: fetch_sales

Prompt: چه تعداد سفارشات امروز دریافت شده است؟
Predicted Intent: fetch_orders

Prompt: کدام محصول بیشترین فروش را داشته است؟
Predicted Intent: fetch_sales

Prompt: درآمد کل این ماه چقدر بوده است؟
Predicted Intent: fetch_revenue

